In [1]:
# drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#%cd /content/drive/MyDrive/Colab Notebooks/Artist_classification/data
#!unzip -qq "/content/drive/MyDrive/Colab Notebooks/Artist_classification/data/artist.zip"

## Import Library

In [3]:
import random
import pandas as pd
import numpy as np
import os
import cv2

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
# device 할당
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [5]:
# Hyperparameter Setting
CFG = {
    'IMG_SIZE': 224,
    'EPOCHS': 100,
    'LEARNING_RATE': 1e-5,
    'BATCH_SIZE': 16,
    'SEED': 41
}

In [6]:
# RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

## Data Preprocessing

In [7]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Artist_classification/data/train.csv')
train.head()

,id,img_path,artist
0,0,./train/0000.jpg,Diego Velazquez
1,1,./train/0001.jpg,Vincent van Gogh
2,2,./train/0002.jpg,Claude Monet
3,3,./train/0003.jpg,Edgar Degas
4,4,./train/0004.jpg,Hieronymus Bosch


In [8]:
def img_path_change(img_path):
  return '/content/drive/MyDrive/Colab Notebooks/Artist_classification/data' + str(img_path)[1:]

train['img_path'] = train['img_path'].apply(img_path_change)

In [9]:
# Label Encoding : artist들을 범주형 데이터로 변환
# 화가 이름 50명
le = preprocessing.LabelEncoder()
train['artist'] = le.fit_transform(train['artist'].values)

## Train / Validation Split

In [10]:
train_df, val_df, _, _ = train_test_split(train, train['artist'].values, test_size=0.2, random_state=CFG['SEED'])

In [11]:
train_df = train_df.sort_values(by=['id'])
train_df.head()

,id,img_path,artist
0,0,/content/drive/MyDrive/Colab Notebooks/Artist_...,9
2,2,/content/drive/MyDrive/Colab Notebooks/Artist_...,7
3,3,/content/drive/MyDrive/Colab Notebooks/Artist_...,10
5,5,/content/drive/MyDrive/Colab Notebooks/Artist_...,38
6,6,/content/drive/MyDrive/Colab Notebooks/Artist_...,43


In [12]:
val_df = val_df.sort_values(by=['id'])
val_df.head()

,id,img_path,artist
1,1,/content/drive/MyDrive/Colab Notebooks/Artist_...,48
4,4,/content/drive/MyDrive/Colab Notebooks/Artist_...,24
17,17,/content/drive/MyDrive/Colab Notebooks/Artist_...,10
21,21,/content/drive/MyDrive/Colab Notebooks/Artist_...,29
29,29,/content/drive/MyDrive/Colab Notebooks/Artist_...,28


## Data Load

In [13]:
# inference=True면 test 데이터라는 뜻.
# 따라서 target에 해당하는 artist를 return할 수 없음.
def get_data(df, infer=False):
  if infer:
    return df['img_path'].values
    
  return df['img_path'].values, df['artist'].values

In [14]:
# 파일 경로, 레이블
train_img_paths, train_labels = get_data(train_df)
val_img_paths, val_labels = get_data(val_df)

In [15]:
train_img_paths[0], train_labels[0]

('/content/drive/MyDrive/Colab Notebooks/Artist_classification/data/train/0000.jpg',
 9)

## CustomDataset

https://wikidocs.net/57165

- 1. 데이터셋을 구성한다.
- 2. 데이터로더를 구성한다.
  - 배치 단위로 모델에 인풋으로 넣어주기 위해 사용.
- 3. 모델에 넣는다.

In [16]:
# torch.utils.data.Dataset이라는 class를 상속받는 자식 클래스
class CustomDataset(Dataset):

  # 데이터셋을 처음 선언할 때, 자동으로 호출.
  # 몇 가지 인수들을 입력받도록 만들 수 있다.
  def __init__(self, img_paths, labels, transforms=None):
    self.img_paths = img_paths
    self.labels = labels
    self.transforms = transforms

  # 데이터셋에서 특정 1개의 샘플을 가져오기
  # index는 몇 번째 데이터를 가져올건지에 대한 변수.
  def __getitem__(self, index):
    img_path = self.img_paths[index]
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # cv2는 BGR순으로 이루어져 있음 -> RGB

    # 아래 dataset 선언을 보면 transform이 사용됨.
    if self.transforms is not None:
      image = self.transforms(image=image)['image']

    if self.labels is not None:
      label = self.labels[index]
      return image, label
    else:
      return image

  # 데이터셋의 길이 (총 샘플의 수)
  # 데이터셋을 선언하고 dataloader를 사용할 때 내부적으로 사용
  ## 데이터셋의 len을 알아야 데이터로더가 미니 배치를 사용할 수 있기 때문
  def __len__(self):
    return len(self.img_paths)

In [17]:
# Albumentation Augmentation
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            # 모든 이미지 데이터의 R, G, B 값 평균, 표준편차
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [18]:
# Data Loader

# num_workers : 데이터 로드 멀티 프로세싱. (몇 개의 서브 프로세스를 사용할 것인지)
# CPU코어를 몇 개 사용할지. num_workers는 CPU.
train_dataset = CustomDataset(train_img_paths, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_img_paths, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [19]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b2(pretrained=True) # b0 ~ b7
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [20]:
#import timm
#class Network(nn.Module):
#    def __init__(self):
#        super(Network, self).__init__()
#        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=88)
#       
#    def forward(self, x):
#        x = self.model(x)
#        return x

## Train

In [21]:
def train(model, optimizer, train_loader, test_loader, scheduler, device):

  # 모델을 device에 할당
  model.to(device)

  # Loss 정의
  criterion = nn.CrossEntropyLoss().to(device)

  # Scheduler에서 사용할 변수 선언
  best_score = 0
  best_model = None

  for epoch in range(1, CFG['EPOCHS'] + 1):
    # model을 train 모드로 전환
    model.train()

    # loss값을 넣을 리스트 생성
    train_loss = []

    # Epoch 진행
    for img, label in tqdm(iter(train_loader)):
      img, label = img.float().to(device), label.to(device)

      # 과거에 이용한 mini batch 내 이미지, 레이블을 바탕으로 계산된 Loss의 Gradient값이 optimizer에 할당되어 있는 것을 방지.
      optimizer.zero_grad()

      # pred값 
      model_pred = model(img)

      # 선언한 Loss에 pred값과 정답을 넣기 
      loss = criterion(model_pred, label)

      # backpropagation
      loss.backward()

      # optimizer
      optimizer.step()

      # loss값 추가
      train_loss.append(loss.item())

    # 최종 loss값 생성
    tr_loss = np.mean(train_loss)

    val_loss, val_score = validation(model, criterion, test_loader, device)

    print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')

    # Scheduler
    if scheduler is not None:
      scheduler.step()

    # val_score을 기준으로 best model 선정
    if best_score < val_score:
      best_model = model
      best_score = val_score

  return best_model

In [22]:
# 이번 대회에서는 F1 score를 사용
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

In [23]:
def validation(model, criterion, test_loader, device):

  # 모델을 평가용으로 전환 (dropout 등의 규제가 들어가지 않게 조절)
  model.eval()

  model_preds = []
  true_labels = []

  val_loss = []

  # 평가 단계에서 Gradient를 통해 파라미터 값이 업데이트되는 현상을 방지
  with torch.no_grad():
    for img, label in tqdm(iter(test_loader)):
      img, label = img.float().to(device), label.to(device)
      model_pred = model(img)
      loss = criterion(model_pred, label)
      val_loss.append(loss.item())

      model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
      true_labels += label.detach().cpu().numpy().tolist()

  val_f1 = competition_metric(true_labels, model_preds)
  return np.mean(val_loss), val_f1

In [ ]:
# Run

model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-bcdf34b7.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-bcdf34b7.pth


  0%|          | 0.00/35.2M [00:00<?, ?B/s]

  0%|          | 0/296 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

Epoch [1], Train Loss : [3.83069] Val Loss : [3.50127] Val F1 Score : [0.05606]


...

Epoch [5], Train Loss : [2.29635] Val Loss : [2.32622] Val F1 Score : [0.24427]


  0%|          | 0/296 [00:00<?, ?it/s]

  0%|          | 0/74 [00:00<?, ?it/s]

Epoch [6], Train Loss : [2.07601] Val Loss : [2.16205] Val F1 Score : [0.27440]


  0%|          | 0/296 [00:00<?, ?it/s]

## Inference

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Artist_classification/data/test.csv')
test.head()

In [ ]:
# Test에는 artist 정보가 없으니 infer=True
test_img_paths = get_data(test, infer=True)

In [ ]:
test_dataset = CustomDataset(test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)
            
            model_pred = model(img)
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
    
    print('Done.')
    return model_preds

In [ ]:
preds = inference(infer_model, test_loader, device)

In [ ]:
preds = le.inverse_transform(preds) # LabelEncoder로 변환 된 Label을 다시 화가이름으로 변환

## Submit

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Artist_classification/data/sample_submission.csv')
submit.head()

In [ ]:
submit['artist'] = preds
submit.head()

In [ ]:
submit.to_csv('./lr1e-5.csv', index=False)